In [1]:
import os
import numpy as np
import librosa.display
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications import MobileNet, ResNet50
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve

2025-03-09 00:01:28.073819: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 00:01:28.081959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741453288.090841  457602 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741453288.093537  457602 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 00:01:28.103855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
NUM_CLASSES = 2 

X = np.load('../SavedFeatures/X_f0.npy')
y = np.load('../SavedFeatures/y_f0.npy')

y_encoded = to_categorical(y, NUM_CLASSES)
split_index = int(0.8 * len(X))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y_encoded[:split_index], y_encoded[split_index:]

X_train = np.expand_dims(X_train, axis=-1) 
X_train = np.tile(X_train, (1, 1, 109)) 

X_val = np.expand_dims(X_val, axis=-1)
X_val = np.tile(X_val, (1, 1, 109))    


In [ ]:
del X,y
gc.collect()

31589

# Modeling

In [32]:
def resnet50_f0(input_shape=(109, 109, 1), num_classes=2):
    base_model = ResNet50(include_top=False, weights=None, input_shape=input_shape)

    x = Flatten()(base_model.output)
    
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

In [33]:
tf.keras.backend.clear_session()

In [ ]:
mrsnt = resnet50_f0(input_shape=(109, 109, 1), num_classes=2)


mrsnt.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


hist = mrsnt.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 53s 42ms/step - accuracy: 0.6487 - loss: 0.7393 - val_accuracy: 0.7340 - val_loss: 0.5983
Epoch 2/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.7475 - loss: 0.5962 - val_accuracy: 0.7443 - val_loss: 0.5378
Epoch 3/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.7578 - loss: 0.5634 - val_accuracy: 0.7622 - val_loss: 0.5300
Epoch 4/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.7646 - loss: 0.5397 - val_accuracy: 0.7952 - val_loss: 0.4741
Epoch 5/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.7815 - loss: 0.5082 - val_accuracy: 0.8068 - val_loss: 0.4529
Epoch 6/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.7989 - loss: 0.4789 - val_accuracy: 0.8034 - val_loss: 0.4541
Epoch 7/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.8117 - loss: 0.4522 - val_accuracy: 0.8084 - val_loss: 0.4533
Epoch 8/10
796/796 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.8011 - loss: 0.4607 - 

In [ ]:
test_loss, test_accuracy = mrsnt.evaluate(X_val, y_val, verbose=0)

y_pred = mrsnt.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 

f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================ResNet50 - F0===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")


eers = []
# Tính EER 
for i in range(y_pred.shape[1]): 
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")



mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
===================ResNet50 - F0===================
TAccuracy: 0.80967
F1-Score: 0.80463
Class 0: EER = 0.20936 at threshold 0.87760
Class 1: EER = 0.20957 at threshold 0.12249
EER: 0.20947


In [42]:
# Save Model as weight with H5 format
mrsnt.save_weights("../SavedModel/F0_mrsnet.weights.h5")

In [43]:

del mrsnt, X_train, X_val, y_train, y_val
gc.collect()


103705